In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

# Open Robert Frost text file and read in data as `text`
with open('drive/My Drive/Colab Notebooks/Robert_Frost.txt', 'r') as f:
    text = f.read()

chars = tuple(set(text)) # creates a tuple containing all unique characters from text
int2char = dict(enumerate(chars)) # Maps each integer to a character to create this dictionary
char2int = {ch: ii for ii, ch in int2char.items()} # Maps each character to an integer (swaps key and value from int2char)

# Encode the text
encoded = np.array([char2int[ch] for ch in text])

# Showing the first 100 encoded characters
encoded[:100]

array([21, 21, 21, 21,  1, 57,  4, 64, 23, 10, 19,  1, 20, 28,  4, 21, 21,
       21, 41, 24, 29, 64, 61, 45, 47, 52, 61, 64, 45,  3,  8, 64,  8, 45,
       64, 66, 31, 22, 33, 52, 64,  8, 42, 22, 64, 49, 33,  6,  8,  3,  9,
       22, 64,  6, 49,  9, 47, 52, 61,  7, 21, 41, 24, 31, 31, 64, 45, 52,
       31, 35, 64,  6,  8, 45, 49, 64,  8, 45, 64,  9, 33, 40, 22, 64,  8,
       42, 22, 64, 31, 22, 33, 36, 22,  6, 64, 33, 11, 33, 35, 21])

In [ ]:
text[:100] #Showing the first 100 characters from Robert_Forst.txt

"\n\n\n\nTHE PASTURE\n\n\nI'm going out to clean the pasture spring;\nI'll only stop to rake the leaves away\n"

In [ ]:
len(chars) # number of unique characters from text

67

In [ ]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot
    
# Defining method to make mini-batches for training
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
          y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
           y[:, :-1], y[:, -1] = x[:, 1:], arr[:,0]
        yield x, y


In [ ]:

# Declaring the model
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers # Keep it to 2 or 3
        self.n_hidden = n_hidden # 256 or 512 is usually good
        self.lr = lr # Try different values between 1e-6 and 1e-1
        
        # creating character dictionaries
        self.chars = tokens 
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        #define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        #define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        #define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        #get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        #pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        out = out.contiguous().view(-1, self.n_hidden)
        
        #put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden

    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [ ]:
# Declaring the train method
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.25, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train() # allows for backpropagation 
    
    opt = torch.optim.Adam(net.parameters(), lr=lr) # Initialize optimizer
    criterion = nn.CrossEntropyLoss() # Define the loss measure
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac)) # 75% of data will be training, 25% will be validation
    data, val_data = data[:val_idx], data[val_idx:]
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                  
                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                net.train()
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [ ]:
# Define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

# Declaring the hyperparameters
batch_size = 128
seq_length = 100
n_epochs =  50# start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.01, print_every=50)


CharRNN(
  (lstm): LSTM(67, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=67, bias=True)
)
Epoch: 9/50... Step: 50... Loss: 2.7365... Val Loss: 2.7371
Epoch: 17/50... Step: 100... Loss: 2.1795... Val Loss: 2.1558
Epoch: 25/50... Step: 150... Loss: 1.8861... Val Loss: 1.9062
Epoch: 34/50... Step: 200... Loss: 1.6277... Val Loss: 1.7988
Epoch: 42/50... Step: 250... Loss: 1.4514... Val Loss: 1.7826
Epoch: 50/50... Step: 300... Loss: 1.3634... Val Loss: 1.8319


In [ ]:
# Defining a method to generate the next character
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
  
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [ ]:
# Declaring a method to generate new text
def sample(net, size, prime='The', top_k=None):
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)
    
# Generating new text
print(sample(net, 1000, prime='A', top_k=5))

All the bristling,
Through the lamp oncight if to see it as too carren?
There she sturt of think of the bill out on a frost
And though there, then he saw home it out in.
What shoulder there and said the was well make a side."

"And think that did him, she's goad
Of as it a like that. If he dirn,
Think it what than I saw the sun some boughs
As the winded with through a shoulders ald stronk that manks."

"In they will see is all as mand
To me thought to look at thought, I shorked their bout.
Where there through the load, and that was to such a lition.
I will starmed from the stove, and loss at the window-poold.
It was burning the wath the lamp to sout.
He says than I say, the sun and shouts and bedinn.
It's not to take it too the long, some terrain
To then all your look and love her sind
Silas took on the sanking a mountain,
But his till you have seided the load, and beside the sun?
I's good of that window this frame of something to tell me where you want?"

"You was tround to light thin